# Verb-basiertes Command-Matching testen

Dieses Notebook testet das **Verb-Matching** mit den neuen CommandTemplates:
- Laden der CommandTemplates (Verb-Listen statt Satz-Beispiele)
- Embedden aller Verb-Formen
- Systematische Tests pro Command mit:
  - User-Sätzen (ganze Sätze!)
  - Verb-Extraktion per spaCy
  - Matching der Verben gegen Verb-Listen
  
**WICHTIG**: Wir testen jetzt den ganzen Flow:
1. User gibt ganzen Satz ein: "geh zum wald"
2. spaCy extrahiert Verb: "gehen"
3. Embedding-Match: "gehen" → Verb-Liste → Command "go"

In [ ]:
import sys
sys.path.append('../src')

# Module neu laden (falls bereits importiert)
import importlib
if 'utils.command_templates' in sys.modules:
    import utils.command_templates
    importlib.reload(utils.command_templates)

from utils.command_templates import COMMAND_TEMPLATES
from sentence_transformers import SentenceTransformer, util
import spacy
import pandas as pd

# NLP-Model laden (für Verb-Extraktion)
# WICHTIG: Nur laden wenn noch nicht geladen (spart RAM!)
if 'nlp' not in globals():
    print("Loading spaCy model...")
    nlp = spacy.load("de_dep_news_trf")
else:
    print("spaCy model already loaded (reusing)")

## 1. Templates laden

In [ ]:
# Übersicht
for template in COMMAND_TEMPLATES:
    print(f"{template.command:10} | {len(template.verbs):2} Verben | Threshold: {template.threshold}")

total_verbs = sum(len(t.verbs) for t in COMMAND_TEMPLATES)
print(f"\nGesamt: {total_verbs} Verb-Formen für {len(COMMAND_TEMPLATES)} Commands")

## 2. Verb-Formen sammeln & embedden

In [ ]:
# Embedding-Model laden
# WICHTIG: Nur laden wenn noch nicht geladen (spart RAM!)
if 'model' not in globals():
    print("Loading Sentence Transformer model...")
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
else:
    print("Sentence Transformer already loaded (reusing)")

# Verb-Formen sammeln
verb_texts = []
verb_commands = []
thresholds = {}

for template in COMMAND_TEMPLATES:
    for verb in template.verbs:
        verb_texts.append(verb)
        verb_commands.append(template.command)
    thresholds[template.command] = template.threshold

print(f"Sammle {len(verb_texts)} Verb-Formen...")

# Embedden
verb_embeddings = model.encode(verb_texts, show_progress_bar=True)

print(f"✓ {len(verb_embeddings)} Embeddings erstellt")
print(f"Shape: {verb_embeddings.shape}")

## 3. Test-Funktionen

### 3.1 Verb-Extraktion

In [ ]:
### 3.2 Command-Matching

def match_verb_to_command(verb):
    """Matched ein extrahiertes Verb gegen die Verb-Listen"""
    if verb is None:
        return None, 0.0, None
    
    verb_emb = model.encode(verb)
    scores = util.cos_sim(verb_emb, verb_embeddings)[0]
    
    best_idx = scores.argmax().item()
    best_score = scores[best_idx].item()
    best_command = verb_commands[best_idx]
    best_verb = verb_texts[best_idx]
    
    return best_command, best_score, best_verb

In [ ]:
def extract_verb(text):
    """Extrahiert das Hauptverb aus einem Satz mit spaCy"""
    doc = nlp(text)
    
    # ROOT-Verb suchen
    for token in doc:
        if token.dep_ == "ROOT":
            return token.lemma_
    
    # Fallback: Erstes Verb
    for token in doc:
        if token.pos_ == "VERB":
            return token.lemma_
    
    return None

In [ ]:
### 3.3 Test-Funktion (kompletter Flow)

def test_command(command_name, test_inputs):
    """Testet Command mit User-Sätzen (ganzer Flow: Satz → Verb → Command)"""
    print(f"\n{'='*100}")
    print(f"TESTING: {command_name.upper()}")
    print(f"{'='*100}\n")
    
    passed = 0
    failed = 0
    
    for query in test_inputs:
        # 1. Verb extrahieren
        extracted_verb = extract_verb(query)
        
        # 2. Command matchen
        best_command, best_score, matched_verb = match_verb_to_command(extracted_verb)
        
        # 3. Evaluieren
        threshold = thresholds.get(best_command, 0.80) if best_command else 0.80
        is_correct = best_command == command_name
        is_pass = best_score >= threshold if best_command else False
        
        if is_correct and is_pass:
            status = "✓ PASS"
            passed += 1
        elif is_correct and not is_pass:
            status = "⚠ LOW"
            failed += 1
        else:
            status = "✗ FAIL"
            failed += 1
        
        # Ausgabe
        verb_info = f"Verb: '{extracted_verb}'" if extracted_verb else "Verb: None"
        print(f"{status} | {best_score:.3f} | '{query}'")
        print(f"         {verb_info} → matched '{matched_verb}' → {best_command}")
        
        if not is_correct:
            print(f"         ❌ Expected: {command_name}")
    
    print(f"\n{'='*100}")
    print(f"SUMMARY: {passed} passed, {failed} failed")
    print(f"{'='*100}\n")

In [ ]:
## 4. Tests durchführen

### 4.1 Test GO

go_tests = [
    # Haupt-Verben
    "geh wald",
    "gehe zum wald",
    "lauf taverne",
    "laufe zur taverne",
    "renn weg",
    "renne zum ausgang",
    
    # Weitere Synonyme
    "spaziere zum see",
    "schlendere zur taverne",
    "eile zum ausgang",
    "hetze weg",
    "flüchte zum wald",
    "fliehe weg",
    
    # Aus Tests (sollten jetzt funktionieren!)
    "stürm zum wald",
    "haste zum ausgang",
    "trabe zum see",
    "schreite zur taverne",
    "pirsch zum lager",
    "stapfe durch wald",
    
    # Komplexe Sätze
    "ich gehe zum wald",
    "geh zur taverne und rede mit wirt",
    "lauf schnell zum ausgang",
]

test_command('go', go_tests)

In [ ]:
### 4.2 Test TAKE

take_tests = [
    # Haupt-Verben
    "nimm schlüssel",
    "nimm den schlüssel",
    "hole beutel",
    "hol dir beutel",
    "pack fackel ein",
    "greif schwert",
    "sammel münzen auf",
    "heb hammer auf",
    
    # Trennbare Verben
    "raff schlüssel auf",
    "schnapp schlüssel",
    "steck schlüssel ein",
    "ergreif schwert",
    
    # Aus Tests (sollten jetzt funktionieren!)
    "fass schlüssel",
    "trag schlüssel",
    "nehm schlüssel mit",
    "beschaff dir schlüssel",
    "birg schlüssel",
    "sicher schlüssel",
    
    # Komplexe Sätze
    "ich nehme schlüssel",
    "nimm schlüssel und öffne truhe",
    "kann ich schlüssel nehmen",
]

test_command('take', take_tests)

In [ ]:
### 4.3 Test USE

use_tests = [
    # Haupt-Verben
    "benutze schlüssel",
    "verwende schlüssel",
    "nutze fackel",
    "öffne truhe",
    "bediene hebel",
    
    # Trennbare Verben
    "aktiviere hebel",
    "betätige hebel",
    "drück hebel",
    "zieh hebel",
    "schließ truhe auf",
    "mach truhe auf",
    
    # Aus Tests (sollten jetzt funktionieren!)
    "entriegle truhe",
    "entsperre truhe",
    "knack truhe",
    "spreng truhe",
    
    # Mit Werkzeug
    "öffne truhe mit schlüssel",
    "benutze schlüssel an truhe",
    
    # Komplexe Sätze
    "ich benutze schlüssel",
    "benutze schlüssel um truhe zu öffnen",
]

test_command('use', use_tests)

In [ ]:
### 4.4 Test EXAMINE

examine_tests = [
    # Haupt-Verben
    "untersuche truhe",
    "betrachte hammer",
    "inspiziere buch",
    "mustere inschrift",
    "prüfe mechanismus",
    "begutachte runen",
    "analysiere karte",
    "schau truhe an",
    
    # Weitere Synonyme
    "check truhe",
    "kontrolliere truhe",
    "beschau truhe",
    "studiere truhe",
    "erforsche truhe",
    
    # Aus Tests (sollten jetzt funktionieren!)
    "scanne truhe",
    "observiere truhe",
    "beobachte truhe",
    "sichte truhe",
    "überprüf truhe",
    "beguck truhe",
    
    # Komplexe Sätze
    "ich untersuche truhe",
    "untersuche truhe genau",
]

test_command('examine', examine_tests)

In [ ]:
### 4.5 Test READ

read_tests = [
    # Haupt-Verben
    "lies buch",
    "lese buch",
    "studiere text",
    "entziffere inschrift",
    
    # Trennbare Verben
    "überfliege buch",
    "durchstöbere buch",
    "dekodiere runen",
    "entschlüssle inschrift",
    "deute runen",
    
    # Aus Tests (sollten jetzt funktionieren!)
    "verschlinge buch",
    "konsultiere buch",
    "schmökere in buch",
    "vertief text",
    "interpretiere text",
    "übersetze inschrift",
    
    # Komplexe Sätze
    "ich lese buch",
    "lies buch um hinweise zu finden",
]

test_command('read', read_tests)

In [ ]:
### 4.6 Test LOOK

look_tests = [
    # Haupt-Verben
    "schau dich um",
    "sieh dich um",
    "guck dich um",
    "schau umher",
    "blick umher",
    
    # Erkundung
    "orientiere mich",
    "kundschafte aus",
    "erkunde umgebung",
    
    # Kurze Formen
    "schau um",
    "schau",
    
    # Aus Tests (sollten jetzt funktionieren!)
    "muster umgebung",
    "sondiere umgebung",
    "inspiziere umgebung",
    "erfass umgebung",
    "scanne raum",
    "überblicke raum",
    
    # Komplexe Sätze
    "ich schaue mich um",
    "schau dich genau um",
]

test_command('look', look_tests)

In [ ]:
### 4.7 Einzelne Verben testen

# Interessante Edge Cases
print("\n" + "="*100)
print("EDGE CASES - Einzelne Verben testen")
print("="*100 + "\n")

edge_cases = [
    ("hole", "take"),
    ("hol", "take"),
    ("fass", "take"),
    ("entriegle", "use"),
    ("knack", "use"),
    ("scan", "look"),
    ("muster", "look"),
]

for verb, expected_cmd in edge_cases:
    cmd, score, matched = match_verb_to_command(verb)
    status = "✓" if cmd == expected_cmd else "✗"
    print(f"{status} | {score:.3f} | '{verb}' → matched '{matched}' → {cmd} (expected: {expected_cmd})")

In [ ]:
## 5. spaCy-Analyse: Was kann das Model?

def analyze_sentence(text):
    """Analysiert einen Satz und zeigt alle spaCy-Informationen"""
    doc = nlp(text)
    
    print(f"\n{'='*100}")
    print(f"SATZ: '{text}'")
    print(f"{'='*100}\n")
    
    print(f"{'Token':<15} {'Lemma':<15} {'POS':<10} {'TAG':<10} {'DEP':<10} {'HEAD':<15} {'Erklärung'}")
    print("-" * 100)
    
    for token in doc:
        explanation = ""
        if token.dep_ == "ROOT":
            explanation = "← HAUPTVERB"
        elif token.pos_ == "VERB":
            explanation = "← Verb"
        elif token.pos_ == "NOUN":
            explanation = "← Nomen"
        elif token.pos_ == "ADJ":
            explanation = "← Adjektiv"
        
        print(f"{token.text:<15} {token.lemma_:<15} {token.pos_:<10} {token.tag_:<10} {token.dep_:<10} {token.head.text:<15} {explanation}")
    
    print("\n")

In [ ]:
# Test-Sätze analysieren - Basis-Beispiele + alle FAILs aus den Tests
test_sentences = [
    # === Basis-Beispiele ===
    "geh zum wald",
    "nimm den schlüssel",
    "öffne die truhe",
    "schau dich um",
    "lies das buch",
    "ich gehe zum wald",
    "geh zur taverne und rede mit wirt",
    
    # === GO FAILs ===
    "spaziere zum see",         # FAIL: Verb 'Spaziere' → matched 'begutachte' → examine
    "flüchte zum wald",         # FAIL: Verb 'flüchen' → matched 'durchleucht' → examine
    
    # === TAKE FAILs ===
    "hole beutel",              # FAIL: Verb 'Beutel' → matched 'beobacht' → examine  ❌ NOMEN!
    "heb hammer auf",           # FAIL: Verb 'heb' → matched 'hast' → go
    "steck schlüssel ein",      # FAIL: Verb 'Steck' → matched 'steige' → go
    "nehm schlüssel mit",       # FAIL: Verb 'nehm' → matched 'gehe' → go
    "birg schlüssel",           # FAIL: Verb 'Schlüssel' → matched 'entschlüsseln' → read  ❌ NOMEN!
    "sicher schlüssel",         # FAIL: Verb 'Schlüssel' → matched 'entschlüsseln' → read  ❌ NOMEN!
    "kann ich schlüssel nehmen", # FAIL: Verb 'können' → matched 'nehme wahr' → look
    
    # === USE FAILs ===
    "bediene hebel",            # FAIL: Verb 'Bediene' → matched 'begutacht' → examine
    "betätige hebel",           # FAIL: Verb 'Hebel' → matched 'sondiere' → look  ❌ NOMEN!
    "zieh hebel",               # FAIL: Verb 'Hebel' → matched 'sondiere' → look  ❌ NOMEN!
    "schließ truhe auf",        # FAIL: Verb 'Schließ' → matched 'stecke ein' → take
    "mach truhe auf",           # FAIL: Verb 'mach' → matched 'klau' → take
    
    # === EXAMINE FAILs ===
    "mustere inschrift",        # FAIL: Verb 'inschrift' → matched 'lese' → read  ❌ NOMEN!
    "scanne truhe",             # FAIL: Verb 'truhe' → matched 'lege um' → use  ❌ NOMEN!
    "sichte truhe",             # FAIL: Verb 'Sichte' → matched 'wahrnehmen' → look
    
    # === READ FAILs ===
    "studiere text",            # FAIL: Verb 'studiere' → matched 'studiere' → examine (Verb-Overlap!)
    
    # === LOOK FAILs ===
    "kundschafte aus",          # FAIL: Verb 'Kundschaft' → matched 'begutachten' → examine  ❌ NOMEN!
    "muster umgebung",          # FAIL: Verb 'Muster' → matched 'mustere' → examine  ❌ NOMEN!
    "inspiziere umgebung",      # FAIL: Verb 'umgebung' → matched 'umlegen' → use  ❌ NOMEN!
    "scanne raum",              # FAIL: Verb 'Raum' → matched 'vertief' → read  ❌ NOMEN!
]

for sentence in test_sentences:
    analyze_sentence(sentence)

## Legende: spaCy Tags & Dependencies

### POS (Part-of-Speech) - Wortarten

| Tag | Bedeutung | Beispiel |
|-----|-----------|----------|
| `VERB` | Verb | gehen, nimm, öffne |
| `NOUN` | Nomen | Wald, Schlüssel, Truhe |
| `ADJ` | Adjektiv | schön, alt, dunkel |
| `ADP` | Präposition | zu, in, auf, an |
| `DET` | Artikel | der, die, das, den |
| `PRON` | Pronomen | ich, du, mich, dich |
| `ADV` | Adverb | schnell, vorsichtig |
| `AUX` | Hilfsverb | kann, wird, ist |
| `CCONJ` | Konjunktion | und, oder, aber |

### TAG (Detaillierte Tags) - STTS (Stuttgart-Tübingen-TagSet)

#### Verben:
| Tag | Bedeutung | Beispiel |
|-----|-----------|----------|
| `VVFIN` | Vollverb, finite Form | geh, nimm, öffne |
| `VVINF` | Vollverb, Infinitiv | gehen, nehmen, öffnen |
| `VVIMP` | Vollverb, Imperativ | geh!, nimm! |
| `VVPP` | Vollverb, Partizip Perfekt | gegangen, genommen |
| `VVIZU` | Vollverb, Infinitiv mit "zu" | zu gehen |
| `VMFIN` | Modalverb, finit | kann, muss, will |

#### Nomen:
| Tag | Bedeutung | Beispiel |
|-----|-----------|----------|
| `NN` | Nomen, Singular/Plural | Wald, Schlüssel, Truhe |
| `NE` | Eigenname | Berlin, Hans |

#### Andere:
| Tag | Bedeutung | Beispiel |
|-----|-----------|----------|
| `ART` | Artikel | der, die, das |
| `APPR` | Präposition | zu, in, auf |
| `PPER` | Personalpronomen | ich, du, er |
| `PRF` | Reflexivpronomen | mich, dich, sich |

### DEP (Dependency) - Syntaktische Abhängigkeiten

| Tag | Bedeutung | Beispiel |
|-----|-----------|----------|
| `ROOT` | Hauptverb des Satzes | "geh" in "geh zum wald" |
| `sb` | Subjekt | "ich" in "ich gehe" |
| `oa` | Akkusativobjekt | "schlüssel" in "nimm schlüssel" |
| `da` | Dativobjekt | "mir" in "gib mir" |
| `mo` | Modifier (Adverbial) | "schnell" in "lauf schnell" |
| `nk` | Attribut/Kern | "den" in "nimm den schlüssel" |
| `op` | Präpositionalobjekt | "wald" in "geh zum wald" |
| `svp` | Separable Verb Prefix | "auf" in "heb auf" |
| `cd` | Koordination | "und" in "geh und nimm" |
| `cj` | Konjunkt | zweites Verb in Koordination |

### Wichtige Erkenntnisse für Verb-Extraktion:

1. **ROOT ist nicht immer ein Verb!** spaCy kann auch Nomen als ROOT markieren
2. **Lemma kann falsch sein**: "flüchte" → "flüchen" statt "flüchten"
3. **Trennbare Verben**: `svp` markiert den abgetrennten Teil
4. **Modalverben**: `VMFIN` sind Hilfsverben, nicht das Hauptverb der Aktion

### Verbesserungs-Strategie:

```python
# Besser: ROOT MUSS ein VERB sein (POS == "VERB")
for token in doc:
    if token.dep_ == "ROOT" and token.pos_ == "VERB":
        return token.lemma_
```

In [ ]:
print("\n" + "="*100)
print("ANALYSE ABGESCHLOSSEN")
print("="*100)